In [20]:
import sklearn.model_selection
import sklearn.tree
import pandas
import sklearn.ensemble
import sklearn.linear_model
import sklearn.svm
import xgboost

In [2]:
path = 'C:\\Users\\Alexey.Osipov\\Downloads\\'
tabHot = pandas.read_csv(path + 'HotStartData.csv', sep = ';')
tabCold = pandas.read_csv(path + 'ColdStartData.csv', sep = ';')

In [3]:
tabHot = pandas.concat([tabHot.drop('GenderInfo', axis=1), pandas.get_dummies(tabHot['GenderInfo'])], axis=1)
tabCold = pandas.concat([tabCold.drop('GenderInfo', axis=1), pandas.get_dummies(tabCold['GenderInfo'])], axis=1)

In [4]:
yHot = tabHot.Class
yCold = tabCold.Class
XHot = tabHot.drop('Class', axis=1)
XCold = tabCold.drop('Class', axis=1)

In [5]:
X_train_Hot, X_test_Hot, y_train_Hot, y_test_Hot = sklearn.model_selection.train_test_split(XHot, yHot, random_state=239)
X_train_Cold, X_test_Cold, y_train_Cold, y_test_Cold = sklearn.model_selection.train_test_split(XCold, yCold, random_state=239)

In [6]:
X_train_Hot.head(), y_train_Hot.head()

(     Duration  CreditHistory  Employment  Guarantors  Age  \
 91   0.666667             -2           4           0   54   
 436  0.333333              1           3           0   23   
 22   0.555556             -2           1           0   48   
 237  1.166667              1           4           0   61   
 848  0.500000              1           3           0   59   
 
      NumberOfCreditsInBank  JobInfo  NumberOfPeopleBeingLiable       Val  \
 91                       1        2                          1 -0.692948   
 436                      1        1                          1 -1.669664   
 22                       2        1                          2 -0.095321   
 237                      2        1                          1  0.176217   
 848                      1        2                          1 -0.734750   
 
      Telephone  Foreign  A91  A92  A93  A94  
 91           0        1    0    0    1    0  
 436          0        1    0    0    0    1  
 22           0      

In [24]:
#sklearn.model_selection.GridSearchCV(sklearn.tree.DecisionTreeClassifier(random_state=239), param_grid=[]
#                                     scoring=sklearn.metrics.average_precision_score, cv = 3)

In [7]:
model = sklearn.tree.DecisionTreeClassifier(random_state=239)
model.fit(X_train_Hot, y_train_Hot)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=239,
            splitter='best')

In [46]:
def modelFit(model, X_train, y_train):
    random_states = [128, 239, 28]
    stat_prec = 0
    stat_brier = 0
    stat_acc = 0
    for random_state in random_states:
        X_1, X_2, y_1, y_2 = sklearn.model_selection.train_test_split(X_train, y_train, random_state = random_state)
        model.fit(X_1, y_1)
        stat_prec += sklearn.metrics.average_precision_score(model.predict(X_2), y_2)
        stat_brier += sklearn.metrics.brier_score_loss(pandas.DataFrame(model.predict_proba(X_2))[1], y_2)
        stat_acc += sklearn.metrics.accuracy_score(model.predict(X_2), y_2)
    return stat_prec/3, stat_acc/3, stat_brier/3

In [47]:
def modelCheck(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    values = model.predict(X_test)
    probs = pandas.DataFrame(model.predict_proba(X_test))
    vals = model.predict(X_test)
    return [sklearn.metrics.average_precision_score(y_test, vals),
    sklearn.metrics.accuracy_score(y_test, vals),
    sklearn.metrics.brier_score_loss(y_test, probs[1])]

In [105]:
print(y_train_Hot.count())

750


In [83]:
#estimate baseline
baselineTrain = []
print(y_train_Hot.count())
for i in range(0, y_train_Hot.count()):
    baselineTrain.append(0)
baselineTest = []
for i in range(0, y_test_Hot.count()):
    baselineTest.append(0)
baselineTrainprobs = []
print(y_train_Hot.count())
for i in range(0, y_train_Hot.count()):
    baselineTrainprobs.append(0.3)
baselineTestprobs = []
for i in range(0, y_test_Hot.count()):
    baselineTestprobs.append(0.3)

print([sklearn.metrics.average_precision_score(y_train_Hot, baselineTrain),
    sklearn.metrics.accuracy_score(y_train_Hot, baselineTrain),
    sklearn.metrics.brier_score_loss(y_train_Hot, baselineTrainprobs)])
print([sklearn.metrics.average_precision_score(y_test_Hot, baselineTest),
    sklearn.metrics.accuracy_score(y_test_Hot, baselineTest),
    sklearn.metrics.brier_score_loss(y_test_Hot, baselineTestprobs)])

750
750
[0.304, 0.696, 0.21159999999999995]
[0.288, 0.712, 0.20519999999999994]


In [48]:
#trees only
modelTree = sklearn.tree.DecisionTreeClassifier(random_state=239)
print(modelFit(modelTree, X_train_Hot, y_train_Hot))
print(modelCheck(modelTree, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

(0.3473673350477533, 0.6365248226950354, 0.36347517730496454)
[0.31444579945799456, 0.616, 0.384]


In [33]:
model = sklearn.tree.DecisionTreeClassifier(max_depth=7, random_state=239)
modelFit(model, X_train_Hot, y_train_Hot)

0.3005570005626001

In [34]:
modelCheck(model, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot)

[0.31599999999999995, 0.676, 0.25160651115054844]

In [140]:
#RF only
modelRF = sklearn.ensemble.RandomForestClassifier(n_estimators=11,random_state=239, max_depth=20,min_samples_leaf=4, min_samples_split=0.1)
print(modelFit(modelRF, X_train_Hot, y_train_Hot))
print(modelCheck(modelRF, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

(0.040454503314328094, 0.6737588652482268, 0.31028368794326244)
[0.330074074074074, 0.728, 0.18871725078334897]


In [141]:
modelRFCold = sklearn.ensemble.RandomForestClassifier(n_estimators=11,random_state=239, max_depth=20,min_samples_leaf=4, min_samples_split=0.1)
print(modelFit(modelRF, X_train_Cold, y_train_Cold))
print(modelCheck(modelRF, X_train_Cold, y_train_Cold, X_test_Cold, y_test_Cold))

(0.05808236325549634, 0.6755319148936171, 0.31560283687943264)
[0.2857361111111111, 0.688, 0.19011163636956832]


In [144]:
(modelRF.feature_importances_), X_train_Hot.columns

(array([0.23820197, 0.14917393, 0.02341734, 0.17343024, 0.01618086,
        0.05331723, 0.04089416, 0.19852072, 0.03420262, 0.01826004,
        0.02292333, 0.02485296, 0.00462944, 0.00199515]),
 Index(['Duration', 'CreditHistory', 'Employment', 'Guarantors', 'Age',
        'NumberOfCreditsInBank', 'JobInfo', 'NumberOfPeopleBeingLiable', 'Val',
        'Telephone', 'Foreign', 'A91', 'A92', 'A93', 'A94'],
       dtype='object'))

In [106]:
from sklearn.tree import export_graphviz

In [108]:
export_graphviz((modelRF.estimators_[0]), out_file = 'tree0.dot', rounded = True, precision = 1)

In [103]:
modelRF.decision_path

<bound method BaseForest.decision_path of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=239, verbose=0, warm_start=False)>

In [36]:
model2 = sklearn.ensemble.RandomForestClassifier(max_depth=3, n_estimators=100, random_state=239)
modelFit(model2, X_train_Hot, y_train_Hot)

0.011916350464535696

In [37]:
modelCheck(model2, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot)

[0.2911111111111111, 0.708, 0.18865254085285665]

In [50]:
model3 = sklearn.linear_model.LogisticRegression(random_state=239)
print(modelFit(model3, X_train_Hot, y_train_Hot))
print(modelCheck(model3, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

(0.11686088753256045, 0.6843971631205674, 0.3120567375886525)
[0.33960714285714283, 0.72, 0.1830218356967652]


[0.3367008547008547, 0.72, 0.18305454945530766]

In [51]:
model4 = sklearn.svm.SVC(probability=True, random_state=239)
print(modelFit(model4, X_train_Hot, y_train_Hot))
print(modelCheck(model4, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

(0.08759938963765174, 0.6861702127659575, 0.3120567375886525)
[0.3094545454545454, 0.716, 0.1975302965049188]


[0.3094545454545454, 0.716, 0.1975302965049188]

In [165]:
model.feature_importances_, X_train_Hot.columns

(array([0.18204604, 0.14207239, 0.09648295, 0.06606861, 0.20870659,
        0.        , 0.01841331, 0.03387637, 0.23740927, 0.        ,
        0.        , 0.        , 0.        , 0.01492447, 0.        ]),
 Index(['Duration', 'CreditHistory', 'Employment', 'Guarantors', 'Age',
        'NumberOfCreditsInBank', 'JobInfo', 'NumberOfPeopleBeingLiable', 'Val',
        'Telephone', 'Foreign', 'A91', 'A92', 'A93', 'A94'],
       dtype='object'))

In [159]:
model.tree_.children_left

array([ 1,  2,  3,  4,  5,  6, -1,  8, -1, -1, 11, -1, -1, 14, 15, 16, -1,
       -1, 19, -1, -1, 22, 23, -1, -1, 26, -1, -1, -1, 30, -1, 32, 33, 34,
       -1, 36, -1, -1, -1, 40, -1, -1, 43, 44, 45, -1, 47, 48, 49, -1, -1,
       52, -1, -1, 55, 56, -1, -1, 59, -1, -1, 62, -1, 64, -1, 66, 67, -1,
       -1, -1, 71, -1, 73, -1, -1], dtype=int64)

In [52]:
modelnew = xgboost.XGBClassifier(max_depth=2, random_state=239, learning_rate=0.01, reg_lambda=5)
print(modelFit(modelnew, X_train_Hot, y_train_Hot))
print(modelCheck(modelnew, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-package

(0.02715254738617275, 0.6808510638297873, 0.31560283687943264)
[0.2966913580246913, 0.708, 0.20155184223576766]


C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
modelnew = xgboost.XGBClassifier(max_depth=2, random_state=239, learning_rate=0.01, reg_lambda=5)
modelCheck(modelnew, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot)

C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Alexey.Osipov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0.2966913580246913, 0.708, 0.20155184223576766]

In [131]:
X_train_Hot.head()

,Duration,CreditHistory,Employment,Guarantors,Age,NumberOfCreditsInBank,JobInfo,NumberOfPeopleBeingLiable,Val,Telephone,Foreign,A91,A92,A93,A94
91,0.666667,-2,4,0,54,1,2,1,-0.692948,0,1,0,0,1,0
436,0.333333,1,3,0,23,1,1,1,-1.669664,0,1,0,0,0,1
22,0.555556,-2,1,0,48,2,1,2,-0.095321,0,0,0,0,1,0
237,1.166667,1,4,0,61,2,1,1,0.176217,0,1,1,0,0,0
848,0.500000,1,3,0,59,1,2,1,-0.734750,0,1,0,0,1,0


In [143]:
#we try to delete:
#(array([0.23820197, 0.14917393, 0.02341734, 0.17343024, 0.01618086,
#        0.05331723, 0.04089416, 0.19852072, 0.03420262, 0.01826004,
#        0.02292333, 0.02485296, 0.00462944, 0.00199515]),
# Index(['Duration', 'CreditHistory', 'Employment', 'Guarantors', 'Age',
#        'NumberOfCreditsInBank', 'JobInfo', 'NumberOfPeopleBeingLiable', 'Val',
#        'Telephone', 'Foreign', 'A91', 'A92', 'A93', 'A94'],
#       dtype='object'))
#gender info -> delete
#Foreign -> delete
#Employment -> delete


In [159]:
X_train_Hot=X_train_Hot.drop('A91', axis=1)
X_train_Hot=X_train_Hot.drop('A92',  axis=1)
X_train_Hot=X_train_Hot.drop('A93',  axis=1)
X_train_Hot=X_train_Hot.drop('A94',  axis=1)
X_train_Hot=X_train_Hot.drop('Foreign',  axis=1)
X_test_Hot=X_test_Hot.drop('A91',  axis=1)
X_test_Hot=X_test_Hot.drop('A92',  axis=1)
X_test_Hot=X_test_Hot.drop('A93',  axis=1)
X_test_Hot=X_test_Hot.drop('A94',  axis=1)
X_test_Hot=X_test_Hot.drop('Foreign',  axis=1)


KeyError: "labels ['A91'] not contained in axis"

In [160]:
X_test_Hot=X_test_Hot.drop('JobInfo',  axis=1)
X_train_Hot=X_train_Hot.drop('JobInfo',  axis=1)

In [161]:
X_train_Hot.columns

Index(['Duration', 'CreditHistory', 'Employment', 'Guarantors', 'Age',
       'NumberOfCreditsInBank', 'NumberOfPeopleBeingLiable', 'Val',
       'Telephone'],
      dtype='object')

In [262]:
#RF only
modelRF = sklearn.ensemble.RandomForestClassifier(n_estimators=23,random_state=239, max_depth=20,min_samples_leaf=4, min_samples_split=0.1)
print(modelFit(modelRF, X_train_Hot, y_train_Hot))
print(modelCheck(modelRF, X_train_Hot, y_train_Hot, X_test_Hot, y_test_Hot))

(0.1034235816255007, 0.6914893617021277, 0.30851063829787234)
[0.30828758169934634, 0.708, 0.18301551382244907]


In [163]:
modelRF.feature_importances_, X_train_Hot.columns

(array([0.24819208, 0.20774852, 0.13678787, 0.01694938, 0.12460749,
        0.0081914 , 0.01714867, 0.22306932, 0.01730526]),
 Index(['Duration', 'CreditHistory', 'Employment', 'Guarantors', 'Age',
        'NumberOfCreditsInBank', 'NumberOfPeopleBeingLiable', 'Val',
        'Telephone'],
       dtype='object'))

In [153]:
#cold case:
X_train_Cold=X_train_Cold.drop('A91', axis=1)
X_train_Cold=X_train_Cold.drop('A92',  axis=1)
X_train_Cold=X_train_Cold.drop('A93',  axis=1)
X_train_Cold=X_train_Cold.drop('A94',  axis=1)
X_train_Cold=X_train_Cold.drop('Foreign',  axis=1)
X_test_Cold=X_test_Cold.drop('A91',  axis=1)
X_test_Cold=X_test_Cold.drop('A92',  axis=1)
X_test_Cold=X_test_Cold.drop('A93',  axis=1)
X_test_Cold=X_test_Cold.drop('A94',  axis=1)
X_test_Cold=X_test_Cold.drop('Foreign',  axis=1)


In [164]:
X_test_Cold=X_test_Cold.drop('JobInfo',  axis=1)
X_train_Cold=X_train_Cold.drop('JobInfo',  axis=1)

In [259]:
#RF only
modelRF2 = sklearn.ensemble.RandomForestClassifier(n_estimators=11,random_state=239, max_depth=5,min_samples_leaf=5, min_samples_split=0.1)
print(modelFit(modelRF2, X_train_Cold, y_train_Cold))
print(modelCheck(modelRF2, X_train_Cold, y_train_Cold, X_test_Cold, y_test_Cold))

(0.04696550378653006, 0.6790780141843972, 0.30851063829787234)
[0.3065802469135802, 0.716, 0.19382510871766315]


In [166]:
modelRF2.feature_importances_, X_train_Cold.columns

(array([0.18813267, 0.15691634, 0.05028441, 0.156325  , 0.02199918,
        0.02443074, 0.35852057, 0.04339109]),
 Index(['Duration', 'Employment', 'Guarantors', 'Age', 'NumberOfCreditsInBank',
        'NumberOfPeopleBeingLiable', 'Val', 'Telephone'],
       dtype='object'))